In [3]:
import os
import pandas as pd
import json

In [6]:
json_dir = 'json'
csv_dir = '../csv'

if not os.path.isdir(csv_dir):
    raise NameError('csv folder doesnt exist')

os.makedirs(json_dir, exist_ok=True)

In [7]:
def get_csv_data(file_name, columns_title):
    file_path = os.path.join(csv_dir, f'{file_name}.csv')
    data = pd.read_csv(file_path, index_col=0)
    data = data.unstack().to_frame()
    data.columns = [columns_title]
    return data

In [8]:
cases = get_csv_data('total_cases', 'cases')
deaths = get_csv_data('total_deaths', 'deaths')
new_cases = get_csv_data('new_cases', 'new_cases')
new_cases_perc = cases.pct_change(axis=1,fill_method='ffill')['new_cases_perc']
new_deaths = get_csv_data('new_deaths', 'new_deaths')

df = cases.join(deaths).join(new_cases).join(new_deaths).join(new_cases_perc)
df = df.reset_index(level=[1])

KeyError: 'new_cases_perc'

In [ ]:
data = {}
for group in df.groupby(level=0):
    data[group[0]] = { 
        'cases': group[1][['date', 'cases']].values.tolist(),
        'deaths': group[1][['date', 'deaths']].values.tolist(),
        'newCases': group[1][['date', 'new_cases']].values.tolist(),
        'newCasesPerc': group[1][['date', 'new_cases_perc']].values.tolist(),
        'newDeaths': group[1][['date', 'new_deaths']].values.tolist(),
    }